# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [13]:
import logging
import csv
import os 
import shutil

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np 
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core import Experiment, Model, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Dataset, Datastore 
from azureml.pipeline.steps import AutoMLStep
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice


print("SDK version: ", azureml.core.VERSION)

SDK version:  1.33.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [11]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart-failure-pred'

experiment=Experiment(ws, experiment_name)
print(experiment)

found = False
ds = 'workspaceblobstore'
key = 'heart-failure-pred'
description_text = "Heart failure prediction dataset for capstone project"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]
if not found:
    data = "https://mlstrg158340.blob.core.windows.net/azureml-blobstore-e1a969f3-969a-4bbd-a15c-43e00c8da638/UI/09-15-2021_091742_UTC/heart_failure_clinical_records_dataset.csv"
    dataset = Dataset.Tabular.from_delimited_files(data)
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

Experiment(Name: heart-failure-pred,
Workspace: quick-starts-ws-158340)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [12]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [16]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                   "max_concurrent_iterations": 5,
                   "primary_metric": "accuracy"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task="classification",
                              compute_target="project-cluster",
                              training_data=dataset,
                              label_column_name="DEATH_EVENT",
                              n_cross_validations=5,
                              **automl_settings)

In [17]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-pred,AutoML_de238e55-cba6-40a2-a7f2-5d64cfbf62ae,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [19]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-pred,AutoML_de238e55-cba6-40a2-a7f2-5d64cfbf62ae,automl,Running,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_de238e55-cba6-40a2-a7f2-5d64cfbf62ae',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-15T10:40:25.868033Z',
 'endTimeUtc': '2021-09-15T10:59:46.358508Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'project-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-pred","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-158340","workspace_name":"quick-starts-ws-158340","region":"southcentralus","compute_target":"project-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metr

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
print(remote_run.get_metrics())

Run(Experiment: heart-failure-pred,
Id: AutoML_de238e55-cba6-40a2-a7f2-5d64cfbf62ae_55,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('randomforestclassifier', RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None, criterion='entropy', max_depth=None, max_features=0.7, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.19052631578947368, min_samples_split=0.5721052631578947, min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False))], flatte

In [21]:
#TODO: Save the best model
model = remote_run.register_model(model_name='hf-pred-best-autml-model')
print(model.name, model.version, model.id, sep='\n')

hf-pred-best-autml-model
1
hf-pred-best-autml-model:1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
from azureml.train.automl.run import AutoMLRun
run_id = "AutoML_de238e55-cba6-40a2-a7f2-5d64cfbf62ae_55"
best_run = AutoMLRun(experiment, run_id)


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-pred,AutoML_de238e55-cba6-40a2-a7f2-5d64cfbf62ae_55,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
script_file = 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
aci_service_name = 'heart-failure-prediction'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

heart-failure-prediction
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-15 11:48:13+00:00 Creating Container Registry if not exists.
2021-09-15 11:48:14+00:00 Registering the environment.
2021-09-15 11:48:15+00:00 Use the existing image.
2021-09-15 11:48:15+00:00 Generating deployment configuration.
2021-09-15 11:48:16+00:00 Submitting deployment to compute.
2021-09-15 11:48:18+00:00 Checking the status of deployment heart-failure-prediction..
2021-09-15 11:58:32+00:00 Checking the status of inference endpoint heart-failure-prediction.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [29]:
aci_service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [36]:
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

In [37]:
import json 
import requests 

test_sample = json.dumps({'data': test_data.to_dict(orient='records')})
response = aci_service.run(input_data=test_sample)
response

'{"result": [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]}'

In [38]:
#testing ground truth

test_label

81     0
143    0
108    0
116    0
18     1
29     1
280    0
222    0
259    0
79     0
Name: DEATH_EVENT, dtype: int64

TODO: In the cell below, print the logs of the web service and delete the service

In [40]:
print(aci_service.get_logs())

2021-09-15T12:05:14,600932000+00:00 - rsyslog/run 
2021-09-15T12:05:14,602683700+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-15T12:05:14,609621800+00:00 - nginx/run 
2021-09-15T12:05:14,622572800+00:00 - iot-server/run 
rsyslogd: /azureml-envs/azureml_77e992fdaf2b83b0c81ee1c90ded8d17/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-15T12:05:15,002977400+00:00 - iot-server/finish 1 0
2021-09-15T12:05:15,005714300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (63)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 91
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-09-15 12:05:17,490 | root | INFO | Starting up app insights client
logging socket was 

In [41]:
aci_service.delete()